In [51]:
import numpy as np

In [52]:
# 逻辑电路设置，X输入向量， W为调节阈值，通过不同的调试可以实现 与或非 和 与非门
def logicalGate(X, W):
    return 1 if np.sum(W * X) > 0 else 0

In [53]:
# 与门 输入电位 减半叠加 和 0.75相比来判断是否为 高低电位
def AND(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([-0.75, 0.5, 0.5]))

In [54]:
# 或门
def OR(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([-0.25, 0.5, 0.5]))

In [55]:
# 非门
def NOT(x1):
    return logicalGate(np.array([1,x1,x1]), np.array([0.75, -0.5, -0.5]))

In [56]:
# 与非门
def NAND(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([0.75, -0.5, -0.5]))

In [57]:
# 异或门
def XOR(x1, x2):
    return AND(OR(x1,x2),NAND(x1,x2))

In [58]:
#测试单目运算
NOT(0),NOT(1)

(1, 0)

In [59]:
# 测试其他的逻辑运算
def testGate(gates) :
    ret = []
    for x1 in range(2):
        for x2 in range(2): 
            item = []
            for gate in gates :
                item.append(gate(x1,x2))
            ret.append(item)
    return ret

In [60]:
def printTestRet(ret):
    print ( 'x1\tx2\tand\tor\tnand\txor' )
    for x1 in range(2):
        for x2 in range(2):
            print(x1, '\t', x2, end ='\t')
            for i in range(4):
                print(ret[x1*2+x2][i], end = '\t' )
            print()

In [61]:
ret = testGate([AND,OR,NAND,XOR])
printTestRet(ret)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [62]:
# 用与非门 实现 与门
def AND2(x1, x2):
    return NAND(NAND(x1,x2), NAND(x1,x2) )

In [63]:
# 用与非门 实现 或门
def OR2(x1, x2):
    return NAND(NAND(x1,x1), NAND(x2,x2) )

In [64]:
# 用与非门 实现 非门
def NOT2(x1):
    return NAND(x1,x1)

In [65]:
# 用与非门 实现 与非门 ，就是本身
def NAND2(x1, x2):
    return NAND(x1, x2 )

In [66]:
# 用与非门 实现 异或门
def XOR2(x1, x2):
    return NAND(NAND(x1, NAND(x2,x2) ), NAND(NAND(x1,x1),x2) )

In [67]:
NOT2(0),NOT2(1)

(1, 0)

In [68]:
ret2 = testGate([AND2,OR2,NAND2,XOR2])
printTestRet(ret2)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [69]:
def XOR1(x1, x2):
    return AND(OR(x1,x2),NOT(AND(x1,x2)))

In [70]:
for xs in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    y = XOR1(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

(0, 0) -> 0
(1, 0) -> 1
(0, 1) -> 1
(1, 1) -> 0


In [71]:
# 半加器的实现
def halfAdd(x1, x2):
    return (XOR(x1, x2), AND(x1, x2))

In [72]:
# 全加器，只能实现一位二进制相加， carryBit 进位
def fullAdd(x1, x2, carryBit):
    s, co = halfAdd(x1, x2)
    s, co2 = halfAdd(carryBit, s)
    return (s, OR(co, co2))

In [73]:
for xs in [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]:
    y = fullAdd(xs[0], xs[1], xs[2])
    print(str(xs) + " -> " + str(y))

(0, 0, 0) -> (0, 0)
(1, 0, 0) -> (1, 0)
(0, 1, 0) -> (1, 0)
(1, 1, 0) -> (0, 1)
(0, 0, 1) -> (1, 0)
(1, 0, 1) -> (0, 1)
(0, 1, 1) -> (0, 1)
(1, 1, 1) -> (1, 1)


In [74]:
# 16位二进制相加， co 为是否进位标志
def add16Bit(a16bit, b16bit, co=0):
    v =[]
    for i in range(16):
        (d, co) = fullAdd(a16bit[i], b16bit[i], co)
        v.append(d)
    return v

In [75]:
# 辅助方法，用数组来表示 16位数
def to16bit(x):
    v16bit =[]
    d = x
    for i in range(16):
        v16bit.append(d % 2)
        d = d // 2
    return v16bit

In [76]:
# 辅助方法，将16位数数组 转换为整数
def from16bit(a16bit):
    d=0
    for i in range(15,-1,-1):
        d = d * 2 + a16bit[i]
    return d
        

In [77]:
# 加法的实现
def add(a, b):
    return from16bit(add16Bit(to16bit(a), to16bit(b)))

In [78]:
add(6,7)

13

In [79]:
# 16位数的补码，就是每一位取反， 但是最低为没有+1
def complement(a16bit):
    for i in range(16):
        a16bit[i] = NOT(a16bit[i])
    return a16bit

In [80]:
# 16位数的减法实现
def sub16Bit(a16bit, b16bit): 
    return add16Bit(a16bit, complement(b16bit), 1)

In [81]:
def sub(a, b):
    return from16bit(sub16Bit(to16bit(a), to16bit(b)))

In [82]:
sub(20, 8)

12

In [83]:
# 模拟 左移位操作
def shitRight(a16bit, shiftBits = 1):
    v16bit =[]
    for i in range(shiftBits):
        v16bit.append(0)
    for i in range(shiftBits, 16):
        v16bit.append(a16bit[i -shiftBits])
    return v16bit

In [84]:
# 模拟 左移位操作
def shiftLeft(a16bit, shiftBits = 1):
    v16bit =[]
    for i in range(16 - shiftBits):
        v16bit.append(a16bit[i+shiftBits])
    for i in range(16 - shiftBits, 16):
        v16bit.append(0)
    return v16bit

In [85]:
from16bit(shiftLeft(to16bit(12))), from16bit(shitRight(to16bit(12)))

(6, 24)

In [95]:
# 逐位与
def AND16BIT1BIT(a16bit, x):
    v16bit =[]
    for i in range(16):
        v16bit.append(AND(a16bit[i], x))
    return v16bit

In [87]:
# 16位二进制 0
def ZERO16BIT():
    v16bit =[]
    for i in range(16):
        v16bit.append(0)
    return v16bit

In [97]:
# 16位数的乘法实现
def multi16Bit(a16bit, b16bit): 
    v16bit = ZERO16BIT()
    for i in range(16):
        v16bit = add16Bit(v16bit, AND16BIT1BIT(a16bit, b16bit[i]))
        a16bit = shitRight(a16bit)
    return v16bit

In [89]:
def multi(a, b):
    return from16bit(multi16Bit(to16bit(a), to16bit(b)))

In [98]:
multi(4, 5)

20